In [1]:
# Importing required libraries

from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime
import time
import smtplib

In [2]:
# Function to send an email notification when product prices drop below budget

def send_email():
    server = smtplib.SMTP('smtp.gmail.com:587')
    server.ehlo()
    server.starttls()
    server.login('from@gmail.com','password')

    subject = "Found the Deal under $70!"
    body = """The price has dropped! 
            Check it out here: 
            Snail Mucin: https://www.ulta.com/p/advanced-snail-96-mucin-power-essence-xlsImpprod15641052?sku=2504912,
            Cleansing Foam: https://www.ulta.com/p/heartleaf-quercetinol-pore-deep-cleansing-foam-pimprod2048920?sku=2634189,
            Serum: https://www.ulta.com/p/madagascar-centella-ampoule-pimprod2045880?sku=2626854"""

    msg = f"Subject: {subject}\n\n{body}"
    
    server.sendmail('from@gmail.com', 'to@gmail.com', msg)

In [3]:
# Function to check product prices, save data to CSV, and trigger email if needed
def price_checker():
    urls = [
        'https://www.ulta.com/p/advanced-snail-96-mucin-power-essence-xlsImpprod15641052?sku=2504912',
        'https://www.ulta.com/p/heartleaf-quercetinol-pore-deep-cleansing-foam-pimprod2048920?sku=2634189',
        'https://www.ulta.com/p/madagascar-centella-ampoule-pimprod2045880?sku=2626854'
    ]

    data = []

    today = datetime.date.today()

    for url in urls:
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html')

        product_info = soup.find('div', class_='ProductInformation')
        product_name = product_info.find('span', class_='Text-ds--title-5').text.strip()

        product_price_info = soup.find('div', class_='ProductPricing')
        product_price = product_price_info.find('span', class_='Text-ds--title-5').text.strip()

        product_price = float(product_price.replace('$', '').strip())

        data.append({
            'Product': product_name,
            'Price': product_price,
            'Date': today
        })

    df = pd.DataFrame(data)

    csv_file = 'product_data.csv'

    df.to_csv(csv_file, index=False)

    print("\nFinal Data:")
    print(df)
    
    total_price = df['Price'].sum()
    print(f"\nToday's Total: ${total_price}")
    
    if(float(total_price) < 70):
        send_email()

In [4]:
# Run the price checker every 12 hours (43200 seconds)
while(True):
    price_checker()
    time.sleep(43200)